# Plot tree trajectories in simple 2D spaces

Trying to identify a space where tree evolution is clear, and types of trees are clear. The spaces should be meaningful.

## 1 Setup

Setup the database

In [1]:
from utils import setup_spreadr
setup_spreadr('spreadr_exp_1')

Imports for the analysis

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np
import seaborn as sb

from gists.models import Sentence, Tree

from utils import grouper, memoized, mpl_palette

Equip models with useful tools

In [3]:
from utils import equip_spreadr_models as utils_equip
utils_equip()

from linguistics import equip_spreadr_models as linguistics_equip
linguistics_equip()

## 2 Exploration